In [4]:
import numpy as np
from flask import Flask, render_template, request
import numpy as np
from keras.models import load_model
import string
import json
import nltk

In [18]:
def read_glove_vecs(glove_file):
    with open(glove_file,encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype='float32')
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

def pp(strr):
    words = strr.split()
    stripped = [w.translate(table) for w in words]
    #stripped = list(filter(None, stripped))
    stripped = [word.lower() for word in stripped]
    #stripped = [w for w in stripped if not w in stop_words]
    #stemmed = [porter.stem(word) for word in stripped]
    return(' '.join(stripped))

def sentences_to_indices(X, word_to_index, max_len):

    m = X.shape[0]                                   # number of training examples
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        sentence_words = X[i].lower().split()
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j+ 1
            
 
    return X_indices

def cosine_similarity(u, v):
  
    distance = 0.0
    dot = np.dot(u,v)
    norm_u = np.linalg.norm(u)
    
    norm_v = np.linalg.norm(v)
    cosine_similarity = dot/np.multiply(norm_u,norm_v)
    
    return cosine_similarity

def sentence_to_avg(sentence, word_to_vec_map):
    words = sentence.lower().split()
    avg = np.zeros((200,))
    for w in words:
        avg += word_to_vec_map[w]
    avg = avg/len(words)  
    return avg

def batao(labl,sent):
    mini = 0
    for i,n in enumerate(dictt[labl]):
            cs = cosine_similarity(n[0],sentence_to_avg(pp(sent),word_to_vec_map))
            if(cs>mini):
                mini = cs
                resp = n[1]
    return(resp)
def chat2():
    print("start chit chat! (Enter quit to exit)")
    while True:
        inp = input("You: ")
        if len(list(inp.split(' ')))>24:
            raise Exception("The length of seentence should be less than 24")
        if inp.lower() == "quit":
            break
        try:
            x_test = np.array([pp(inp)])
            X_test_indices = sentences_to_indices(x_test, word_to_index, 24)
            pred = model.predict(X_test_indices)
            num = np.argmax(pred)
            lebl = labllist[num]
            print(batao(lebl,inp))
        except TypeError:
            print("NULL value")
        except:
            print("please use ")
        

In [6]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove6b200d/glove.6B.200d.txt')
model = load_model('chatter.h5')

W1128 08:22:04.887823 10872 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1128 08:22:05.421366 10872 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1128 08:22:05.452355 10872 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1128 08:22:06.467751 10872 deprecation_wrapper.py:119] From D:\MiniConda\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1128 08:22:06.474719 10872 de

In [7]:
table = str.maketrans('', '', string.punctuation)

In [8]:
with open("intents.json") as file:
    data = json.load(file)

In [9]:
words = []
labels = []
docs_x = []
docs_y = []
xx = []
for intent in data["intents"]:
        for pattern in intent["patterns"]:
            ##stemming to eliminate extra words
            ##tokeninze
            wrds = nltk.word_tokenize(pattern) ##list of words in patters
            words.extend(wrds) ##instead of append
            xx.append(pattern)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])
            
labels = sorted(labels)      

In [10]:
fasa=[]
for line in xx:
    words = line.split()
    stripped = [w.translate(table) for w in words]
    stripped = [word.lower() for word in stripped]
    fasa.append(' '.join(stripped))
X_train = [i for i in fasa if i]

In [11]:
reps = []
for intent in data["intents"]:
        for responses in intent["responses"]:
            reps.append(responses)

In [12]:
dictt = {}
for i,n in enumerate(X_train):
    if not docs_y[i] in dictt.keys():
        dictt[docs_y[i]]=[]
    dictt[docs_y[i]].append([sentence_to_avg(pp(n),word_to_vec_map),reps[i]])

In [13]:
def predict(inp):
    x_test = np.array([pp(inp)])
    X_test_indices = sentences_to_indices(x_test, word_to_index, 24)
    pred = model.predict(X_test_indices)
    num = np.argmax(pred)
    lebl = labllist[num]
    return (batao(lebl,inp))

In [14]:
labllist = np.unique(np.array(docs_y))

In [23]:
app = Flask(__name__)

@app.route('/')
def index():
    appDict = {"ans":"oh my god!"}
    app_json = json.dumps(appDict)
    return appDict

@app.route("/test_connection/in=<inpp>")
def test_connection(inpp):
    #ino=request.args.get(inp)
    ans = {"ans":predict(inpp)}
    return json.dumps(ans)

@app.route("/indo")
def indo():
	return render_template("WebPage.html")

In [24]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


I1128 08:26:33.326798 10872 _internal.py:122]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
I1128 08:26:36.441755 12972 _internal.py:122] 127.0.0.1 - - [28/Nov/2019 08:26:36] "GET /test_connection/in=how%20much%20do%20i%20need%20to%20pay HTTP/1.1" 200 -
I1128 08:26:48.105258  4804 _internal.py:122] 127.0.0.1 - - [28/Nov/2019 08:26:48] "GET /test_connection/in=where%20can%20i%20pay HTTP/1.1" 200 -
I1128 08:27:55.240793  6324 _internal.py:122] 127.0.0.1 - - [28/Nov/2019 08:27:55] "GET /test_connection/in=hi HTTP/1.1" 200 -
I1128 08:31:38.702854  6016 _internal.py:122] 127.0.0.1 - - [28/Nov/2019 08:31:38] "GET /test_connection/in=hi HTTP/1.1" 200 -
I1128 08:32:03.160336  7968 _internal.py:122] 127.0.0.1 - - [28/Nov/2019 08:32:03] "GET /test_connection/in=hi HTTP/1.1" 200 -
I1128 08:32:54.794818  5928 _internal.py:122] 127.0.0.1 - - [28/Nov/2019 08:32:54] "GET /test_connection/in=hi HTTP/1.1" 200 -
I1128 08:33:13.189020 12684 _internal.py:122] 127.0.0.1 - - [28/Nov/2019 08:33

In [21]:
chat2()

start chit chat! (Enter quit to exit)
You: hi
Hello, thanks for visiting
You: how much do i need to pay?
Go to your student portal under the MyConcordia Menu choose "Financial » Students Account". Your student account will indicate the amount due each term and the deadline date by which your payment must appear on your student account.
You: quit


In [2]:
app = Flask(__name__)

@app.route('/')
def index():
	return render_template("home.html")

In [22]:
predict("how much do i need to pay?")

'Go to your\xa0student portal\xa0under the MyConcordia Menu choose "Financial » Students Account". Your student account will indicate the amount due each term and the deadline date by which your payment must appear on your student account.'